**Install requirements**

In [24]:
"""!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'"""

"!pip3 install 'torch==1.3.1'\n!pip3 install 'torchvision==0.5.0'\n!pip3 install 'Pillow-SIMD'\n!pip3 install 'tqdm'"

**Import libraries**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

**Set arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
DATA_DIR = 'DATA' # here the dataset will be downloaded

NUM_CLASSES = 100 

# @toupdate the following vals (look at icarl paper)

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Retrieving dataset CIFAR1000**

In [36]:
# Clone github repository with dataset handler
!rm -r Cifar100/ #debug purposes
if not os.path.isdir('./Cifar100'):
  !git clone https://github.com/danielegenta/Progetto-MLDL.git
  !mv 'Progetto-MLDL' 'Cifar100'
  !rm -r Cifar100/Theoretical-Sources
  !rm -rf Cifar100/ProjectMLDL.ipynb



Cloning into 'Progetto-MLDL'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 75 (delta 22), reused 59 (delta 14), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [37]:
from Cifar100.Dataset.cifar100 import CIFAR100
# Download dataset from the official sourse and save it into DATA
!wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
!tar -xf 'cifar-100-python.tar.gz'  
!mv 'cifar-100-python' $DATA_DIR
!rm -rf 'cifar-100-python.tar.gz'

--2020-05-15 14:48:02--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  49.3MB/s    in 3.7s    

2020-05-15 14:48:06 (44.0 MB/s) - ‘cifar-100-python.tar.gz’ saved [169001437/169001437]

mv: cannot move 'cifar-100-python' to 'DATA/cifar-100-python': Directory not empty


**Define data preprocessing**

In [0]:
# it is ok to use also .5 mean and .5 std (faq1)
# @tocheck
# ref: https://github.com/chengyangfu/pytorch-vgg-cifar10/blob/master/main.py + pytorch resnet documentation
# Define transformations for training
train_transform = transforms.Compose([transforms.Resize(32), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transformations for evaluation
eval_transform = transforms.Compose([transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                                   
])

**Prepare dataset**

In [39]:
from random import shuffle

# Import dataset
train_dataset = CIFAR100(DATA_DIR, split='train', transform=train_transform)
test_dataset = CIFAR100(DATA_DIR, split='test', transform=eval_transform)

# @todo
# split into train, test, 
print(len(train_dataset))
print(len(test_dataset))

## debug: both sets have 100 classes
#print(len(train_dataset.getTargets()))
#print(len(test_dataset.getTargets()))
##

# output a dictionary containing
# group_i: 10 unordered classes
# 10 groups = 100 classes
def createClassesGroups(all_classes):
  dictionary = {}
  group = 1
  shuffle(all_classes)
  j = 0
  for i in range(1,11):
    subgroup = all_classes[j:i*10]
    dictionary[i] = subgroup
    j = i*10
  return dictionary

# create 10 groups of 10 classes each
all_classes = train_dataset.getTargets()
dictionaryClassesGroups = createClassesGroups(list(all_classes))
#print(dictionaryClassesGroups) # debug

# TEST
# given a group, load a small training set containing the corresponding classes
# ex. debug purposes => group1
classes1 = dictionaryClassesGroups[1]
idx = train_dataset.get_indices(classes1)
class1_imgs = Subset(train_dataset, idx)
print(len(class1_imgs))

50000
10000
5000


In [ ]:
splits = train_dataset.split_in_train_val_groups(ratio=0.5, seed=30)

**Prepare dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloaders = []
val_dataloaders = []
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=False)

for v in splits.values():
    train_subs = Subset(train_dataset, v['train'])
    val_subs = Subset(train_dataset, v['val'])

    train_dl = DataLoader(train_subs, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
    val_dl = DataLoader(train_subs, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=False)

    train_dataloaders.append(train_dl)
    val_dataloaders.append(val_dl)
